In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

/Users/seyeong/anaconda3/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [5]:
data = np.load('datasets/datasets/Classical/65_MachineryFault.npz', allow_pickle=True)
X, y = data['X'], data['y']

In [33]:
X[:5]

array([[ 4.4359   , -1.1073   ,  0.22269  ,  0.05398  ,  0.39697  ,
        -0.037787 ,  0.11994  , -0.14026  ],
       [ 4.4628   ,  0.47153  ,  0.049814 ,  0.020857 ,  0.43099  ,
        -0.037627 ,  0.14496  ,  0.15596  ],
       [ 4.4511   , -1.8167   ,  0.054311 ,  0.0053652,  0.38292  ,
        -0.037447 ,  0.10992  , -0.045535 ],
       [ 4.445    ,  0.97556  ,  0.28069  ,  0.02315  ,  0.45651  ,
        -0.031581 ,  0.19738  ,  0.029962 ],
       [ 4.4661   , -1.77     , -0.18746  , -0.089957 ,  0.37851  ,
        -0.033218 ,  0.162    ,  0.07596  ]])

In [34]:
y[:5]

array([0., 0., 0., 0., 0.])

In [6]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [14]:
# 데이터셋 분할 (학습 데이터와 테스트 데이터)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


In [19]:
import tensorflow as tf
print(tf.__version__)


2.15.0


In [27]:
X_train.shape[1]


8

In [29]:
y_train

(1800000,)

In [9]:
# LSTM 모델 정의
model = Sequential()
model.add(LSTM(units=50, input_shape=(X_train.shape[1], 1), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(units=1, activation='sigmoid'))



In [10]:
# 모델 컴파일
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [13]:
# 모델 학습
model.fit(X_train.reshape((X_train.shape[0], X_train.shape[1], 3)), y_train, epochs=10, batch_size=32, validation_data=(X_test.reshape((X_test.shape[0], X_test.shape[1], 3)), y_test))


ValueError: cannot reshape array of size 14400000 into shape (1800000,8,3)

In [11]:
# 예측
y_pred = model.predict(X_test.reshape((X_test.shape[0], X_test.shape[1], 1)))


14063/14063 [==============================] - 13s 880us/step


In [13]:
# 결과 출력
print(y_pred)   

[[0.01705359]
 [0.9999998 ]
 [0.99999994]
 ...
 [0.00313937]
 [0.99989545]
 [0.9999999 ]]


In [18]:
# 이진 클래스로 다시 변환
y_pred_prob = model.predict(X_test.reshape((X_test.shape[0], X_test.shape[1], 1)))
y_pred = (y_pred_prob > 0.8).astype(int)


14063/14063 [==============================] - 12s 853us/step


In [19]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, y_pred.flatten())
print(f'Accuracy: {accuracy:.2f}')

Accuracy: 0.95
